## This notebook explains what happens exactly in the BuildingLoader function and Autosize function

### 1. Import all needed libraries

In [47]:
# Run this again after editing submodules so Colab uses the updated versions
from citylearn import  CityLearn
import matplotlib.pyplot as plt
from pathlib import Path
from agent import RL_Agents_Coord
import numpy as np                                                                                                                                                                                      
import csv
import time
import re
import pandas as pd
import torch
from joblib import dump, load
import json
from energy_models import HeatPump, ElectricHeater, EnergyStorage, Building
import gym
from gym import spaces

### 2. Load environment data

In [48]:
# Load environment
climate_zone = 1
data_path = Path("data/Climate_Zone_"+str(climate_zone))
building_attributes = data_path / 'building_attributes.json'
weather_file = data_path / 'weather_data.csv'
solar_profile = data_path / 'solar_generation_1kW.csv'
building_state_actions = 'buildings_state_action_space.json'
building_ids = ["Building_1","Building_2","Building_3","Building_4","Building_5","Building_6","Building_7","Building_8","Building_9"]
objective_function = ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','quadratic']

In [49]:
with open(building_attributes) as json_file:
    data = json.load(json_file)

This is just a print statement to check the data which is in the available in the building_attributes file.
The data corresponds to a dictionary per building user id (uid) and the information wrt storage devices, heat pump,
electric water heater, solar generation etc.

In [50]:
for uid, attributes in zip(data, data.values()):
    print(uid)
    print(attributes)

Building_1
{'File_Name': 'Building_1.csv', 'File_Name_Solar_Profile': 'solar_generation_1kW.csv', 'Building_Type': 1, 'Climate_Zone': 1, 'Solar_Power_Installed(kW)': 120, 'Heat_Pump': {'nominal_power': 'autosize', 'technical_efficiency': 0.2, 't_target_heating': 45, 't_target_cooling': 8}, 'Electric_Water_Heater': {'nominal_power': 'autosize', 'efficiency': 0.9}, 'Chilled_Water_Tank': {'capacity': 3, 'loss_coefficient': 0.006}, 'DHW_Tank': {'capacity': 3, 'loss_coefficient': 0.008}}
Building_2
{'File_Name': 'Building_2.csv', 'File_Name_Solar_Profile': 'solar_generation_1kW.csv', 'Building_Type': 2, 'Climate_Zone': 1, 'Solar_Power_Installed(kW)': 0, 'Heat_Pump': {'nominal_power': 'autosize', 'technical_efficiency': 0.21, 't_target_heating': 45, 't_target_cooling': 9}, 'Electric_Water_Heater': {'nominal_power': 'autosize', 'efficiency': 0.92}, 'Chilled_Water_Tank': {'capacity': 3, 'loss_coefficient': 0.006}, 'DHW_Tank': {'capacity': 3, 'loss_coefficient': 0.008}}
Building_3
{'File_Name':

### 3. Auto_size function which is used in the next section, get back to this section when the function is called
Automatically size the nominal power of the heat pump and electric heater. Also size the storage capacity of the storage devices. It assumes fixed target temperatures of the heat pump for heating and cooling, which combines with weather data to estimate their hourly COP for the simulated period. The HeatPump is sized such that it will always be able to fully satisfy the heating and cooling demands of the  building. This function also sizes the EnergyStorage devices, setting their capacity as 3 times the maximum hourly  cooling demand in the simulated period (capacity is set in the building_attributes file).

In [51]:
def auto_size(buildings):
    
    # Loop over all the buildings
    for building in buildings.values():
        
        # Autosize guarantees that the DHW device is large enough to always satisfy the maximum DHW demand
        if building.dhw_heating_device.nominal_power == 'autosize':
            
            # If the DHW device is a HeatPump
            if isinstance(building.dhw_heating_device, HeatPump):
                
                #We assume the heat pump is always large enough to meet the highest heating or cooling demand of the building
                building.dhw_heating_device.nominal_power = np.array(building.sim_results['dhw_demand'] \
                                                                     /building.dhw_heating_device.cop_heating).max()
                
            # If the device is an electric heater
            elif isinstance(building.dhw_heating_device, ElectricHeater):
                building.dhw_heating_device.nominal_power = (np.array(building.sim_results['dhw_demand']) \
                                                             /building.dhw_heating_device.efficiency).max()
        
        # Autosize guarantees that the cooling device device is large enough to always satisfy the maximum DHW demand
        if building.cooling_device.nominal_power == 'autosize':

            building.cooling_device.nominal_power = (np.array(building.sim_results['cooling_demand']) \
                                                     /building.cooling_device.cop_cooling).max()
        
        # Defining the capacity of the storage devices as a number of times the maximum demand
        building.dhw_storage.capacity = max(building.sim_results['dhw_demand'])*building.dhw_storage.capacity
        building.cooling_storage.capacity = max(building.sim_results['cooling_demand'])*building.cooling_storage.capacity
        
        # Done in order to avoid dividing by 0 if the capacity is 0
        if building.dhw_storage.capacity <= 0.00001:
            building.dhw_storage.capacity = 0.00001
        if building.cooling_storage.capacity <= 0.00001:
            building.cooling_storage.capacity = 0.00001

### 4. Loop over all buildings which are present in the building_attributes json file and do the following for each:
1. We create instances of the heat pump, electric water heater, chilled water tank and dhw tank 
2. Create a building instance by using the instances as attributes
3. Open the simulation data for the corresponding building and fill the building with its corresponding data
4. Open the weather data for the corresponding building and fill the building with its corresponding data
5. Reading the building attributes
6. Reading the building solar generation profile
7. Finding the max and min values of all the states, which can then be used by the RL agent to scale the states and train any function approximators more effectively
8. Finding the max and min values of all the actions (after scaling them by the capacity), which can then be used by the RL agent to scale the states and train any function approximators more effectively
9. Setting the state and action space for the corresponding building
10. Append the observation space and action space of each building to a general list which contains all the observation and action spaces of the buildings together
11. Add the building to the general dictionary with all the buildings
12. Create an observation and action space for the central_agent.
13. Loop over the buildings and if they have a HeatPump as dhw_heating_device, the Coefficient of Performance is being calculated
14. Autosize the nominal power of the heat/cooling devices and the capacity of the storage devices

In [52]:
save_memory = True # Variable which indicates if we want to save memory or not

#dictionary with all the created buildings and lists with the corresponding observation and action space per building
buildings, observation_spaces, action_spaces = {},[],[]

# Lists which have the min/max values for each state and action option of each building.
# Appended_states will have the values for the weather which are common for all buildings
# These values will then be used to scale the inputs to the function approximator
s_low_central_agent, s_high_central_agent, appended_states = [], [], []
a_low_central_agent, a_high_central_agent, appended_actions = [], [], []

# Loop over all the buildings and their corresponding attributes
for uid, attributes in zip(data, data.values()):
    if uid in building_ids:
        #1) Construct the heat pump, electric water, chilled water tank and dwh tank objects
        heat_pump = HeatPump(nominal_power = attributes['Heat_Pump']['nominal_power'], 
                             eta_tech = attributes['Heat_Pump']['technical_efficiency'], 
                             t_target_heating = attributes['Heat_Pump']['t_target_heating'], 
                             t_target_cooling = attributes['Heat_Pump']['t_target_cooling'], save_memory = save_memory)

        electric_heater = ElectricHeater(nominal_power = attributes['Electric_Water_Heater']['nominal_power'], 
                                         efficiency = attributes['Electric_Water_Heater']['efficiency'], save_memory = save_memory)

        chilled_water_tank = EnergyStorage(capacity = attributes['Chilled_Water_Tank']['capacity'],
                                           loss_coeff = attributes['Chilled_Water_Tank']['loss_coefficient'], save_memory = save_memory)

        dhw_tank = EnergyStorage(capacity = attributes['DHW_Tank']['capacity'],
                                 loss_coeff = attributes['DHW_Tank']['loss_coefficient'], save_memory = save_memory)
        
        #2) Create the corresponding building instance
        building = Building(buildingId = uid, dhw_storage = dhw_tank, cooling_storage = chilled_water_tank, dhw_heating_device = electric_heater, cooling_device = heat_pump, save_memory = save_memory)
        
        #3) Open the simulation data for the corresponding building and fill the building with its corresponding data
        data_file = str(uid) + '.csv'
        simulation_data = data_path / data_file
        with open(simulation_data) as csv_file:
            data = pd.read_csv(csv_file)
            
        building.sim_results['cooling_demand'] = list(data['Cooling Load [kWh]'])
        building.sim_results['dhw_demand'] = list(data['DHW Heating [kWh]'])
        building.sim_results['non_shiftable_load'] = list(data['Equipment Electric Power [kWh]'])
        building.sim_results['month'] = list(data['Month'])
        building.sim_results['day'] = list(data['Day Type'])
        building.sim_results['hour'] = list(data['Hour'])
        building.sim_results['daylight_savings_status'] = list(data['Daylight Savings Status'])
        building.sim_results['t_in'] = list(data['Indoor Temperature [C]'])
        building.sim_results['avg_unmet_setpoint'] = list(data['Average Unmet Cooling Setpoint Difference [C]'])
        building.sim_results['rh_in'] = list(data['Indoor Relative Humidity [%]'])
        
        #4) Open the weather data for the corresponding building and fill the building with its corresponding data
        with open(weather_file) as csv_file:
            weather_data = pd.read_csv(csv_file)
                
        building.sim_results['t_out'] = list(weather_data['Outdoor Drybulb Temperature [C]'])
        building.sim_results['rh_out'] = list(weather_data['Outdoor Relative Humidity [%]'])
        building.sim_results['diffuse_solar_rad'] = list(weather_data['Diffuse Solar Radiation [W/m2]'])
        building.sim_results['direct_solar_rad'] = list(weather_data['Direct Solar Radiation [W/m2]'])

        building.sim_results['t_out_pred_6h'] = list(weather_data['6h Prediction Outdoor Drybulb Temperature [C]'])
        building.sim_results['t_out_pred_12h'] = list(weather_data['12h Prediction Outdoor Drybulb Temperature [C]'])
        building.sim_results['t_out_pred_24h'] = list(weather_data['24h Prediction Outdoor Drybulb Temperature [C]'])

        building.sim_results['rh_out_pred_6h'] = list(weather_data['6h Prediction Outdoor Relative Humidity [%]'])
        building.sim_results['rh_out_pred_12h'] = list(weather_data['12h Prediction Outdoor Relative Humidity [%]'])
        building.sim_results['rh_out_pred_24h'] = list(weather_data['24h Prediction Outdoor Relative Humidity [%]'])

        building.sim_results['diffuse_solar_rad_pred_6h'] = list(weather_data['6h Prediction Diffuse Solar Radiation [W/m2]'])
        building.sim_results['diffuse_solar_rad_pred_12h'] = list(weather_data['12h Prediction Diffuse Solar Radiation [W/m2]'])
        building.sim_results['diffuse_solar_rad_pred_24h'] = list(weather_data['24h Prediction Diffuse Solar Radiation [W/m2]'])

        building.sim_results['direct_solar_rad_pred_6h'] = list(weather_data['6h Prediction Direct Solar Radiation [W/m2]'])
        building.sim_results['direct_solar_rad_pred_12h'] = list(weather_data['12h Prediction Direct Solar Radiation [W/m2]'])
        building.sim_results['direct_solar_rad_pred_24h'] = list(weather_data['24h Prediction Direct Solar Radiation [W/m2]'])
        
        #5) Reading the building attributes
        building.building_type = attributes['Building_Type']
        building.climate_zone = attributes['Climate_Zone']
        building.solar_power_capacity = attributes['Solar_Power_Installed(kW)']
        
        #6) Reading the building solar generation profile
        with open(solar_profile) as csv_file:
                data = pd.read_csv(csv_file)

        building.sim_results['solar_gen'] = list(attributes['Solar_Power_Installed(kW)']*data['Hourly Data: AC inverter power (W)']/1000)
        
        #7) Finding the max and min values of all the states, which can then be used by the RL agent to scale the states 
        # and train any function approximators more effectively
        s_low, s_high = [], []
        
        # Read in the buildings_states_actions data
        with open(building_state_actions) as json_file:
            buildings_states_actions = json.load(json_file)
        
        # Loop over the states and their correspond True/False values
        for state_name, value in zip(buildings_states_actions[uid]['states'], buildings_states_actions[uid]['states'].values()):
            # if the value is true, we append its min and max values to the lists s_low, s_high and s_low_central_agent and s_high_central_agent
            if value == True:
                if state_name == "net_electricity_consumption":
                    # lower and upper bound of net electricity consumption are rough estimates and may not be super accurate. 
                    # Scaling this state-variable using these bounds may result in normalized values above 1 or below 0.
                    # Not sure on the last 2 terms that are added? Neither about the division by 0.8 and 2
                    _net_elec_cons_upper_bound = max(np.array(building.sim_results['non_shiftable_load']) \
                                                     - np.array(building.sim_results['solar_gen']) \
                                                     + np.array(building.sim_results['dhw_demand'])/.8 \
                                                     + np.array(building.sim_results['cooling_demand']) \
                                                     + building.dhw_storage.capacity/.8 \
                                                     + building.cooling_storage.capacity/2)
                    s_low.append(0.)
                    s_high.append(_net_elec_cons_upper_bound)

                    s_low_central_agent.append(0.)
                    s_high_central_agent.append(_net_elec_cons_upper_bound)

                elif state_name != 'cooling_storage_soc' and state_name != 'dhw_storage_soc':
                    s_low.append(min(building.sim_results[state_name]))
                    s_high.append(max(building.sim_results[state_name]))

                    # Create boundaries of the observation space of a centralized agent (if a central agent is being used instead of 
                    # decentralized ones). We include all the weather variables used as states, and use the list appended_states to make sure 
                    # we don't include any repeated states (i.e. weather variables measured by different buildings)
                    if state_name in ['t_in', 'avg_unmet_setpoint', 'rh_in', 'non_shiftable_load', 'solar_gen']:
                        s_low_central_agent.append(min(building.sim_results[state_name]))
                        s_high_central_agent.append(max(building.sim_results[state_name]))

                    elif state_name not in appended_states:
                        s_low_central_agent.append(min(building.sim_results[state_name]))
                        s_high_central_agent.append(max(building.sim_results[state_name]))
                        appended_states.append(state_name)
                else:
                    s_low.append(0.0)
                    s_high.append(1.0)
                    s_low_central_agent.append(0.0)
                    s_high_central_agent.append(1.0)
                    
        #8) Finding the max and min values of all the actions (after scaling them with their capacity), which can then be 
        # used by the RL agent to scale the actions and train any function approximators more effectively. 
        
        '''The energy storage (tank) capacity indicates how many times bigger the tank is compared to the maximum hourly 
        energy demand of the building (cooling or DHW respectively), which sets a lower bound for the action of 1/tank_capacity,
        as the energy storage device can't provide the building with more energy than it will ever need for a given hour. 
        The heat pump is sized using approximately the maximum hourly energy demand of the building (after accounting for the 
        COP, see function autosize). Therefore, we make the fair assumption that the action also has an upper bound equal to 
        1/tank_capacity. This boundaries should speed up the learning process of the agents and make them more stable rather 
        than if we just set them to -1 and 1. I.e. if Chilled_Water_Tank Capacity is 3 (3 times the max. hourly demand of the 
        building in the entire year), its actions will be bounded between -1/3 and 1/3'''
        
        # Loop over the actions and their corresponding True/False values
        a_low, a_high = [], []    
        for action_name, value in zip(buildings_states_actions[uid]['actions'], buildings_states_actions[uid]['actions'].values()):
            if value == True:
                # Only two action names are possible which are cooling_storage and dhw_storage
                if action_name =='cooling_storage':

                    # Avoid division by 0
                    if attributes['Chilled_Water_Tank']['capacity'] > 0.000001:                            
                        a_low.append(max(-1.0/attributes['Chilled_Water_Tank']['capacity'], -1.0))
                        a_high.append(min(1.0/attributes['Chilled_Water_Tank']['capacity'], 1.0))
                        a_low_central_agent.append(max(-1.0/attributes['Chilled_Water_Tank']['capacity'], -1.0))
                        a_high_central_agent.append(min(1.0/attributes['Chilled_Water_Tank']['capacity'], 1.0))
                    else:
                        a_low.append(-1.0)
                        a_high.append(1.0)
                        a_low_central_agent.append(-1.0)
                        a_high_central_agent.append(1.0)
                else:
                    if attributes['DHW_Tank']['capacity'] > 0.000001:
                        a_low.append(max(-1.0/attributes['DHW_Tank']['capacity'], -1.0))
                        a_high.append(min(1.0/attributes['DHW_Tank']['capacity'], 1.0))
                        a_low_central_agent.append(max(-1.0/attributes['DHW_Tank']['capacity'], -1.0))
                        a_high_central_agent.append(min(1.0/attributes['DHW_Tank']['capacity'], 1.0))
                    else:
                        a_low.append(-1.0)
                        a_high.append(1.0)
                        a_low_central_agent.append(-1.0)
                        a_high_central_agent.append(1.0)
        
        # 9) Setting the state and action space for the corresponding building
        building.set_state_space(np.array(s_high), np.array(s_low))
        building.set_action_space(np.array(a_high), np.array(a_low))
        
        # 10) Append the observation space and action space of each building to a general list which contains all the
        # observation and action spaces of the buildings together
        observation_spaces.append(building.observation_space)
        action_spaces.append(building.action_space)
        
        # 11) Add the building to the general dictionary with all the buildings
        buildings[uid] = building
        
        # 12) Create an observation and action space for the central_agent. Box means you're dealing with real valued quantities.
        # The first array are the lowest accepted values while the second array is the highest accepted values.
        # Check the following link to see how it looks like
        #https://stackoverflow.com/questions/44404281/openai-gym-understanding-action-space-notation-spaces-box
        
        observation_space_central_agent = spaces.Box(low=np.float32(np.array(s_low_central_agent)), 
                                                     high=np.float32(np.array(s_high_central_agent)), 
                                                     dtype=np.float32)
        action_space_central_agent = spaces.Box(low=np.float32(np.array(a_low_central_agent)), 
                                                high=np.float32(np.array(a_high_central_agent)), 
                                                dtype=np.float32)
       
        # 13) Loop over the buildings and if they have a HeatPump as dhw_heating_device, the Coefficient of Performance
        # is being calculated
        for building in buildings.values():

            # If the DHW device is a HeatPump
            if isinstance(building.dhw_heating_device, HeatPump):

                # Calculating COPs of the heat pumps for every hour (check the paper for this calculation)
                building.dhw_heating_device.cop_heating = building.dhw_heating_device.eta_tech \
                                                            * (building.dhw_heating_device.t_target_heating + 273.15) \
                                                            / (building.dhw_heating_device.t_target_heating \
                                                            - weather_data['Outdoor Drybulb Temperature [C]'])
                
                # Make sure the COP is not below 0 and not above 20, not sure why these boundaries are chosen?
                building.dhw_heating_device.cop_heating[building.dhw_heating_device.cop_heating < 0] = 20.0
                building.dhw_heating_device.cop_heating[building.dhw_heating_device.cop_heating > 20] = 20.0
                building.dhw_heating_device.cop_heating = building.dhw_heating_device.cop_heating.to_numpy()
            
            # All buildings use the heatpump for cooling so here we don't have to check the instance
            building.cooling_device.cop_cooling = building.cooling_device.eta_tech \
                                                    *(building.cooling_device.t_target_cooling + 273.15) \
                                                    /(weather_data['Outdoor Drybulb Temperature [C]'] \
                                                    - building.cooling_device.t_target_cooling)
            
            # Make sure the COP is not below 0 and not above 20, not sure why these boundaries are chosen?
            building.cooling_device.cop_cooling[building.cooling_device.cop_cooling < 0] = 20.0
            building.cooling_device.cop_cooling[building.cooling_device.cop_cooling > 20] = 20.0
            building.cooling_device.cop_cooling = building.cooling_device.cop_cooling.to_numpy()
            
            # Reset the building instance such that it can be filled again in the next iteration
            building.reset()
        
        # 14) autosize the heat pumps and the storage devices
        # Automatically sizes the heat pumps and the storage devices. It assumes fixed target temperatures of the heat pump 
        # for heating and cooling, which combines with weather data to estimate their hourly COP for the simulated period. 
        # The HeatPump is sized such that it will always be able to fully satisfy the heating and cooling demands of the 
        # building. This function also sizes the EnergyStorage devices, setting their capacity as 3 times the maximum hourly 
        # cooling demand in the simulated period (capacity is set in the building_attributes file).
        auto_size(buildings)
        
        # The function normally returns the following variables which are then later on used for the next steps
        # buildings, observation_spaces, action_spaces, observation_space_central_agent, action_space_central_agent

### 5. Inspect some of the variables which are returned by the function

Get the max and min values for the observation and action spaces of a certain building

In [71]:
print(observation_spaces[0].high)
print(observation_spaces[0].low)
print(action_spaces[0].high)
print(action_spaces[0].low)

[1.2000000e+01 8.0000000e+00 2.4000000e+01 3.5540001e+01 3.5650002e+01
 3.5770000e+01 3.6110001e+01 1.0000000e+02 1.0000000e+02 1.0000000e+02
 1.0000000e+02 4.9356000e+02 4.9714001e+02 5.0534000e+02 5.2245001e+02
 1.0111200e+03 1.0165400e+03 1.0305100e+03 1.0584700e+03 2.6340000e+01
 6.7459999e+01 7.0910004e+01 9.9675598e+01 1.0000000e+00 1.0000000e+00
 3.2535483e+02]
[ 1.    1.    1.   -1.37 -1.55 -1.77 -2.2  29.75 29.57 29.39 28.26  0.
  0.    0.    0.    0.    0.    0.    0.   17.97 21.13  7.44  0.    0.
  0.    0.  ]
[0.33333334 0.33333334]
[-0.33333334 -0.33333334]


### Building class
Arguments
* buildingId (int)
* dhw_storage (EnergyStorage)
* cooling_storage (EnergyStorage)
* electrical_storage (EnergyStorage)
* dhw_heating_device (ElectricHeater or HeatPump)
* cooling_device (HeatPump)

In [99]:
print(buildings['Building_1'].__dict__)

{'building_type': 1, 'climate_zone': 1, 'solar_power_capacity': 120, 'buildingId': 'Building_1', 'dhw_storage': <energy_models.EnergyStorage object at 0x000002050204AF08>, 'cooling_storage': <energy_models.EnergyStorage object at 0x000002050204AE48>, 'electrical_storage': None, 'dhw_heating_device': <energy_models.ElectricHeater object at 0x000002050204AB08>, 'cooling_device': <energy_models.HeatPump object at 0x000002050204A908>, 'observation_space': Box(26,), 'action_space': Box(2,), 'time_step': 0, 'sim_results': {'cooling_demand': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.44, 67.72, 53.95, 63.05, 69.78, 53.72, 44.32, 15.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1, 0.34, 9.83, 17.22, 22.35, 29.34, 32.81, 32.48, 29.61, 26.9, 23.67, 16.92, 9.37, 2.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31, 14.08, 32.74, 50.09, 61.76, 71.41, 70.21, 66.53, 57.98, 45.72, 39.33, 35.99, 30.28, 25.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Get some of the attributes from a building instance

In [98]:
print(buildings['Building_1'].buildingId)
print(buildings['Building_1'].building_type)
print(buildings['Building_1'].climate_zone)
print(buildings['Building_1'].solar_power_capacity)

print(buildings['Building_1'].dhw_storage)
print(buildings['Building_1'].cooling_storage)
print(buildings['Building_1'].electrical_storage) #is not yet incorporated in the CityLearn package

print(buildings['Building_1'].dhw_heating_device)
print(buildings['Building_1'].cooling_device)
print(buildings['Building_1'].observation_space)
print(buildings['Building_1'].observation_space.low)
print(buildings['Building_1'].observation_space.high)
print(buildings['Building_1'].action_space)
print(buildings['Building_1'].action_space.low)
print(buildings['Building_1'].action_space.high)
print(buildings['Building_1'].time_step)
#print(buildings['Building_1'].sim_results)
print(buildings['Building_1'].sim_results.keys())
print(buildings['Building_1'].save_memory)

Building_1
1
1
120
None
Box(26,)
[ 1.    1.    1.   -1.37 -1.55 -1.77 -2.2  29.75 29.57 29.39 28.26  0.
  0.    0.    0.    0.    0.    0.    0.   17.97 21.13  7.44  0.    0.
  0.    0.  ]
[1.2000000e+01 8.0000000e+00 2.4000000e+01 3.5540001e+01 3.5650002e+01
 3.5770000e+01 3.6110001e+01 1.0000000e+02 1.0000000e+02 1.0000000e+02
 1.0000000e+02 4.9356000e+02 4.9714001e+02 5.0534000e+02 5.2245001e+02
 1.0111200e+03 1.0165400e+03 1.0305100e+03 1.0584700e+03 2.6340000e+01
 6.7459999e+01 7.0910004e+01 9.9675598e+01 1.0000000e+00 1.0000000e+00
 3.2535483e+02]
Box(2,)
[-0.33333334 -0.33333334]
[0.33333334 0.33333334]
0
dict_keys(['cooling_demand', 'dhw_demand', 'non_shiftable_load', 'month', 'day', 'hour', 'daylight_savings_status', 't_in', 'avg_unmet_setpoint', 'rh_in', 't_out', 'rh_out', 'diffuse_solar_rad', 'direct_solar_rad', 't_out_pred_6h', 't_out_pred_12h', 't_out_pred_24h', 'rh_out_pred_6h', 'rh_out_pred_12h', 'rh_out_pred_24h', 'diffuse_solar_rad_pred_6h', 'diffuse_solar_rad_pred_12h

Get some of the attributes from a specific Storage or heating/cooling device in a particular building

In [90]:
devices = {"heating device": buildings['Building_1'].dhw_heating_device, 
           "cooling device": buildings['Building_1'].cooling_device, 
           "dhw storage": buildings['Building_1'].dhw_storage, 
           "cooling storage": buildings['Building_1'].cooling_storage
          }

More information on the devices in the buildings:
### Heatpump class
Arguments:
* nominal_power (float): Maximum amount of electric power that the heat pump can consume from the power grid (given by thenominal power of the compressor)* eta_tech (float): Technical efficiency
* t_target_heating (float): Temperature at which the heating energy is released
* t_target_cooling (float): Temperature at which the cooling energy is released

### Electric heater class
Arguments:
* nominal_power (float): Maximum amount of electric power that the electric heater can consume from the power grid
* efficiency (float): efficiency

### Energy storage class
Generic energy storage class. It can be used as a chilled water storage tank or a DHW storage tank
Arguments:
* capacity (float): Maximum amount of energy that the storage unit is able to store (kWh)
* max_power_output (float): Maximum amount of power that the storage unit can output (kW)
* max_power_charging (float): Maximum amount of power that the storage unit can use to charge (kW)
* efficiency (float): Efficiency factor of charging and discharging the storage unit (from 0 to 1)
* loss_coeff (float): Loss coefficient used to calculate the amount of energy lost every hour (from 0 to 1)


Look into the class attributes

In [94]:
for device_type, device_object in zip(devices, devices.values()):
    print(device_type)
    print(device_object.__class__.__name__)
    print(device_object.__dict__)
    #print("\n")
    print("-"*120)

heating device
ElectricHeater
{'nominal_power': 7.322222222222222, 'efficiency': 0.9, 'max_heating': None, 'electrical_consumption_heating': [], '_electrical_consumption_heating': 0, 'heat_supply': [], 'time_step': 0, 'save_memory': True}
------------------------------------------------------------------------------------------------------------------------
cooling device
HeatPump
{'nominal_power': 146.55511648586165, 'eta_tech': 0.2, 'max_cooling': None, 'max_heating': None, '_cop_heating': None, '_cop_cooling': None, 't_target_heating': 45, 't_target_cooling': 8, 't_source_heating': None, 't_source_cooling': None, 'cop_heating': [], 'cop_cooling': array([ 5.73190622,  6.90786241,  6.94197531, ..., 17.68238994,
       20.        , 20.        ]), 'electrical_consumption_cooling': [], 'electrical_consumption_heating': [], 'heat_supply': [], 'cooling_supply': [], 'time_step': 0, 'save_memory': True, '_electrical_consumption_cooling': 0, '_electrical_consumption_heating': 0}
-------------